# Show responses of energy balance model

In [ ]:
import os

from fair.energy_balance_model import EnergyBalanceModel
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np

In [ ]:
cmip6_output_df = pd.read_csv('../data/calibration/v1.4.1/4xCO2_cmip6.csv')
cmip6_output_df.head()

In [ ]:
# where models submit more than one simulation, we choose the one that looks the most stable
multi_runs = {
    "GISS-E2-1-G": "r1i1p1f1",
    "GISS-E2-1-H": "r1i1p3f1",
    "MRI-ESM2-0": "r1i1p1f1",
    "EC-Earth3": "r3i1p1f1",
    "FIO-ESM-2-0": "r1i1p1f1",
    "CanESM5": "r1i1p2f1",
    "FGOALS-f3-L": "r1i1p1f1",
    "CNRM-ESM2-1": "r1i1p1f2",
}

In [ ]:
models = cmip6_output_df.climate_model.unique()

In [ ]:
tas_cmip6 = {}
for model in models:
    if model in multi_runs:
        tas_cmip6[model] = cmip6_output_df.loc[
            (cmip6_output_df['climate_model']==model) & 
            (cmip6_output_df['member_id']==multi_runs[model]) & 
            (cmip6_output_df['variable']=='tas'), 
            'X1850':'X1999'
        ].values.squeeze()
    else:
        tas_cmip6[model] = cmip6_output_df.loc[
            (cmip6_output_df['climate_model']==model) & 
            (cmip6_output_df['variable']=='tas'), 
            'X1850':'X1999'
        ].values.squeeze()

In [ ]:
ebm_fits_df = pd.read_csv('../data/calibration/v1.4.1/4xCO2_cummins_ebm3_cmip6.csv')
ebm_fits_df.head()

In [ ]:
# run 150 year 4xCO2 with the calibrations
ebm3 = {}
for isp, model in enumerate(models):
    print(model)
    if model in multi_runs:
        params = ebm_fits_df.loc[
            (ebm_fits_df['model']==model) &
            (ebm_fits_df['run']==multi_runs[model]),
            'gamma':
        ]
    else:
        params = ebm_fits_df.loc[ebm_fits_df['model']==model, 'gamma':]
    ebm3[model] = EnergyBalanceModel(
        ocean_heat_capacity=[params["C1"].values[0], params["C2"].values[0], params["C3"].values[0]],
        ocean_heat_transfer=[params["kappa1"].values[0], params["kappa2"].values[0], params["kappa3"].values[0]],
        deep_ocean_efficacy=params["epsilon"].values[0],
        gamma_autocorrelation=params["gamma"].values[0],
        sigma_xi=params["sigma_xi"].values[0],
        sigma_eta=params["sigma_eta"].values[0],
        forcing_4co2=params["F_4xCO2"].values[0],
        stochastic_run=True,
        seed=10000 * isp + 700  # reproducibility, but a different stochastic seed per model
    )
    ebm3[model].add_forcing(np.ones(151) * params["F_4xCO2"].values[0], timestep=1)
    ebm3[model].run()

In [ ]:
ebm3["GISS-E2-1-H"].temperature

In [ ]:
# font.size           : 12
# font.family         : Arial
# ytick.direction     : in
# ytick.minor.visible : True
# ytick.major.right   : True
# ytick.right         : True
# xtick.direction     : in
# xtick.minor.visible : True
# xtick.major.top     : True
# xtick.top           : True
# axes.spines.top     : True
# axes.spines.bottom  : True
# figure.dpi          : 150
pl.rcParams['font.size'] = 7
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.major.top'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['xtick.direction'] = 'in'
pl.rcParams['ytick.direction'] = 'in'

In [ ]:
fig, ax = pl.subplots(7, 7, figsize=(18/2.54, 18/2.54))
for isp, model in enumerate(sorted(models)):
    row = isp//7
    col = isp%7
    label1 = 'CMIP6' if model=='UKESM1-0-LL' else ''
    label2 = 'FaIR' if model=='UKESM1-0-LL' else ''
    ax[row, col].plot(np.arange(0.5, 150), tas_cmip6[model], color='r', label=label1)
    ax[row, col].plot(np.arange(151), ebm3[model].temperature[:, 0], color='k', label=label2)
    ax[row, col].set_xlim(0,150)
    ax[row, col].set_ylim(0, 10)
    ax[row, col].text(75, 8.5, model, fontsize=6, ha='center', va='bottom', fontweight='bold')
    ax[row, col].set_xticks([0, 50, 100, 150])
    ax[row, col].set_yticks([0, 2, 4, 6, 8, 10])
    if row<6 or col%2==1:
        ax[row, col].set_xticklabels([])
    if col>0 or row%2==1:
        ax[row, col].set_yticklabels([])
ax[6,3].set_xlabel('Year')
ax[3,0].set_ylabel('°C')
ax[6,6].legend(loc='lower right', frameon=False)
fig.tight_layout()
pl.subplots_adjust(wspace=0, hspace=0)

os.makedirs('../plots/', exist_ok=True)
pl.savefig('../plots/ebm3_150.png')
pl.savefig('../plots/ebm3_150.pdf')